In [1]:
pip install -r ../../requirements.txt


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [3]:
%run -i ../../zkstats/core.py

In [4]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
# this is just dummy random value
sel_dummy_data_path = os.path.join('shared/sel_dummy_data.json')

=======================  ZK-STATS FLOW =======================

In [5]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

create_dummy(data_path, dummy_data_path)

In [6]:
scales = [8]
selected_columns = ['col_name']
commitment_maps = get_data_commitment_maps(data_path, scales)

In [7]:
# Verifier/ data consumer side: send desired calculation
from zkstats.computation import computation_to_model, State


def computation(s: State, data: list[torch.Tensor]) -> torch.Tensor:
    x = data[0]
    # FIXME: should be replaced by `s.where` when it's available. Now the result may be incorrect
    filter = (x > 20)
    filtered_x = torch.where(filter, x, 1.0)
    return s.geometric_mean(filtered_x)

error = 0.01
_, verifier_model = computation_to_model(computation, error)

verifier_define_calculation(dummy_data_path, selected_columns, sel_dummy_data_path,verifier_model, verifier_model_path)

/Users/mhchia/projects/work/pse/zk-stats-lib/zkstats/computation.py:166: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  is_precise_aggregated = torch.tensor(1.0)
/Users/mhchia/projects/work/pse/zk-stats-lib/zkstats/ops.py:102: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  return torch.abs((torch.log(self.result)*size)-torch.sum(torch.log(x)))<=size*torch.log(torch.tensor(1+self.error))
/Users/mhchia/Library/Caches/pypoetry/virtualenvs/zkstats-brXmXluj-py3.12/lib/python3.12/site-

In [8]:
# Prover/ data owner side
_, prover_model = computation_to_model(computation, error)

prover_gen_settings(data_path, selected_columns, sel_data_path, prover_model, prover_model_path, scales, "resources", settings_path)

==== Generate & Calibrate Setting ====
scale:  [8]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":8,"param_scale":8,"scale_rebase_multiplier":10,"lookup_range":[-2560,45056],"logrows":16,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":14432,"total_assignments":3310,"total_const_size":305,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,8],"model_input_scales":[8],"module_sizes":{"kzg":[],"poseidon":[14432,[1]],"elgamal":[0,[0]]},"required_lookups":["Abs",{"Ln":{"scale":256.0}},{"GreaterThan":{"a":0.0}}],"check_mode":"UNSAFE","version":"7.0.0","num_blinding_factors":null}


In [9]:
# Here verifier & prover can concurrently call setup since all params are public to get pk.
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

spawning module 0


==== setting up ezkl ====


spawning module 2
spawning module 0
spawning module 2


Time setup: 13.265241861343384 seconds
Theory output:  tensor(48.1276, dtype=torch.float64)
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 45.703125
==== Generating Proof ====


spawning module 0
spawning module 2


proof:  {'instances': [[[11768814371718170976, 435173728250646979, 519717007263840094, 1741290966923863957], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [14342780245625474618, 733556062893518525, 16161524522791859482, 2634936898687523127]]], 'proof': '2dbd7a643e148997ba065e1bf38df01ad8bd5e91213501c3134a7a6039fd39601224f125455f2a971aa96e8f3214517f3468643ca72b11c7812ce832220b59fc302f5eea0f1d58d6a12af654da221470714662bfc9a6979d19865336afd9c6610489b66559a0227c9e16e840ab8b667e32148abf691b8b54e2d21f5bc3d15c4e1eaa3ab74d4aaf5ca556487d5e3ce0e182f129d8e288c11f832abf6906833e011e6139dc4739de6f491194eb3b1d5dbc822a8b5daeb2b78c28223ade9c04323d24f185c53757b4037163ca59472d4efb0d3bfe139d704effcd948ee40c987dbc081b8f195ff532650cb1c171938681ac3e21d4112d4b6562d046edaf0d383b3324713121c201e8a6ff15461c4cc8212ff8b5acb0d7a431c0e6cfa3c1c7e663bc1f68afca3628af06cc6af7d023d1fa06042fe22ee50071718a73feeb5efaf42d1b4bf261134887ae0c6b5fa95a4c4b799b5601c7cccb62a5a1a0025e58b09a942

In [10]:
# Verifier verifies
res = verifier_verify(proof_path, settings_path, vk_path, selected_columns, commitment_maps)
print("Verifier gets result:", res)

Verifier gets result: 45.703125
